In [1]:
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import os
import pandas as pd
import random
import math
from selenium.webdriver.support.ui import Select
from selenium.common.exceptions import NoSuchElementException
import logging

path = '/Users/victorwu/Documents/EZ Messenger'
os.chdir(path)

In [10]:
class Block:
    
    def __init__(self, df, email, k):
        df = df[df['plastiq_email']==email]
        self.record = df
        self.length = len(df)
        self.starting_index = df.index.min()
        
        self.email = df['plastiq_email'].iloc[k]
        self.password = df['plastiq_password'].iloc[k]
        self.payee_merchant_name = df['payee_merchant_name'].iloc[k]
        self.amount = df['amount'].iloc[k]
        self.account_holder_name = df['recipient_name'].iloc[k]

In [11]:
def try_to_get(statement):
    try:
        statement
        return statement
        logging.info('FOUND %s' % statement)
    except:
        return 'NOT FOUND'
        logging.info('NOT FOUND %s' % statement)

In [12]:
def get_recipient_id(unique_string, ez_messenger_recipients):
    return ez_messenger_recipients[ez_messenger_recipients['unique_string']==unique_string]['recipient_id'].iloc[0]
    
    
def get_plastiq_email(unique_string, ez_messenger_recipients):
    return ez_messenger_recipients[ez_messenger_recipients['unique_string']==unique_string]['plastiq_email'].iloc[0]
    

def get_plastiq_password(unique_string, ez_messenger_recipients):
    return ez_messenger_recipients[ez_messenger_recipients['unique_string']==unique_string]['plastiq_password'].iloc[0]   


def get_payee_merchant_name(unique_string, ez_messenger_recipients):
    return ez_messenger_recipients[ez_messenger_recipients['unique_string']==unique_string]['payee_merchant_name'].iloc[0]   


def is_recipient_in_db(unique_string, ez_messenger_recipients):
    return ez_messenger_recipients[ez_messenger_recipients['unique_string']==unique_string]['is_recipient_in_db'].iloc[0]  


In [13]:
def get_elem(xpath, element_name, wait):
    try:
        elem = WebDriverWait(browser, wait).until(EC.element_to_be_clickable((By.XPATH, xpath)))
        logging.info('%s element variablized' % (element_name))
        return elem
    except Exception as e:
        logging.error('Failed to find %s: %s' % (element_name, e))

In [14]:
def send_keys(elem, keys):
    elem.clear()
    elem.send_keys(str(keys))

In [15]:
def scroll_up():
    body = browser.find_element_by_tag_name('body')
    body.click()
    body.send_keys(Keys.CONTROL + Keys.HOME)

In [16]:
def submit_payment(submission_type):
    if submission_type == 'SCHEDULED':
            submit_payment_elem = get_elem(submit_scheduled_payment_xpath, 'SUBMIT SCHEDULED PAYMENT', 60)
            scroll_up()
            time.sleep(3)
            submit_payment_elem = get_elem(submit_scheduled_payment_xpath, 'SUBMIT SCHEDULED PAYMENT', 20)
            submit_payment_elem.click()
            logging.info('Click SUBMIT SCHEDULED PAYMENT')
    if submission_type =='IMMEDIATE':
        try:
            submit_payment_elem = get_elem(submit_immediate_payment_xpath, 'SUBMIT IMMEDIATE PAYMENT', 60)
            scroll_up()
            time.sleep(3)
            submit_payment_elem = get_elem(submit_immediate_payment_xpath, 'SUBMIT IMMEDIATE PAYMENT', 20)
            submit_payment_elem.click()
            logging.info('Click SUBMIT IMMEDIATE PAYMENT')
        except:
            submission_type = 'SCHEDULED'
            submit_payment_elem = get_elem(submit_scheduled_payment_xpath, 'SUBMIT SCHEDULED PAYMENT', 60)
            scroll_up()
            time.sleep(3)
            submit_payment_elem = get_elem(submit_scheduled_payment_xpath, 'SUBMIT SCHEDULED PAYMENT', 20)
            submit_payment_elem.click()
            logging.info('Click SUBMIT SCHEDULED PAYMENT')
            
        
        

### Read in Recipient Master List

In [17]:
ez_messenger_recipients = pd.read_excel('EZ Messenger Master Recipient List.xlsx',
              names = ['recipient_id', 'recipient_name', 'routing_number',
                        'account_number', 'unique_string', 'type', 'payee_merchant_name',
                        'plastiq_email', 'plastiq_password', 'is_recipient_in_db', 'street_address_1',
                            'street_address_2', 'city', 'state', 'postal_code', 'country', 'phone'],
        converters={'routing_number': lambda x: str(x), 'account_number': lambda x: str(x),
                   'phone': lambda x: str(x)})

### Read in Payments File

In [18]:
#Read in payments file. By this step, all new recipients should already be identified and appended to Master List
payroll_payments = pd.read_csv('EZ ACH File 2-28-19 Payroll.csv',
                       names=['recipient_name', 'routing_number', 'account_number', 'type', 'amount',
                             'street_address_1', 'street_address_2', 'city', 'state', 'postal_code', 'country', 'phone'],
                            converters={'routing_number': lambda x: str(x), 'account_number': lambda x: str(x),
                                       'phone': lambda x: str(x)})

### Scrub Payments Data

In [19]:
payroll_payments['unique_string'] = payroll_payments['recipient_name'] + payroll_payments['routing_number'].astype(str) + payroll_payments['account_number'].astype(str)

In [20]:
#Get list of recipients
old_recipient_list = ez_messenger_recipients['unique_string'].tolist()

In [21]:
#Apply recipient_id to payments 
payroll_payments['recipient_id'] = payroll_payments['unique_string'].apply(lambda x: get_recipient_id(x, ez_messenger_recipients))

In [22]:
#Apply plastiq_email to payments
payroll_payments['plastiq_email'] = payroll_payments['unique_string'].apply(lambda x: get_plastiq_email(x, ez_messenger_recipients))

In [23]:
#Apply plastiq_password to payments
payroll_payments['plastiq_password'] = payroll_payments['unique_string'].apply(lambda x: get_plastiq_password(x, ez_messenger_recipients))

In [24]:
#Get the payee_merchant_name
payroll_payments['payee_merchant_name'] = payroll_payments['unique_string'].apply(lambda x: get_payee_merchant_name(x, ez_messenger_recipients))

In [25]:
#Get column to determine if recipient is in db or not
payroll_payments['is_recipient_in_db'] = payroll_payments['unique_string'].apply(lambda x: is_recipient_in_db(x, ez_messenger_recipients))

In [26]:
payroll_payments['is_payment_submitted'] = 0

In [27]:
#Filter for checking account payments only
payroll_payments = payroll_payments[payroll_payments['type']==1]

### Create Indexing

In [28]:
#Sort payments by email and recipient id
payroll_payments = payroll_payments.sort_values(by=['plastiq_email', 'recipient_id'])

In [29]:
plastiq_email_list = payroll_payments['plastiq_email'].unique().tolist()
plastiq_password_list = payroll_payments['plastiq_password'].unique().tolist()

In [30]:
#Reset index
payroll_payments.reset_index(drop=True, inplace=True)

### Fill in variable fields

In [33]:
#Fill in variable fields here!!!
invoice_line = 'Payment for work completed by 2/28/19'
calendar_date = '6'
submission_type = 'IMMEDIATE'

plastiq_user_email_xpath = '//*[@id="email"]'
plastiq_user_password_xpath = '//*[@id="password-hidden"]'
start_payment_xpath = '//*[@id="pnrStartPaymentButton"]'
recipient_xpath = ''
payment_details_xpath = '//*[@id="paymentAmountSubmitButton"]'
arrival_date_xpath = '//*[@id="app-body"]/div/div[3]/div[2]/div/div[3]/form/div/div[2]/div/div/pq-calendar-input/div/div'
calendar_date_xpath = ''
payment_amount_xpath = '//*[@id="amount"]'
schedule_dropdown_xpath = '//*[@id="dropdown"]'
one_time_schedule_xpath = '//*[@id="dropdownHtmlOption4"]'
continue_payment_xpath = '//*[@id="paymentDetailsContinueButton"]'
accountholder_name_xpath = '//*[@id="payer_fullname"]'
invoice_number_xpath = '//*[@id="payer_account"]'
memo_xpath = '//*[@id="payer_memo"]'
invoice_upload_success_xpath = '//*[@id="app-body"]/div/div[3]/div[2]/div/div[3]/form/div/pq-file-upload/div/div[1]/div[1]/div[1]/div[1]/span[2]'
submit_immediate_payment_xpath = '//*[@id="pmt-review-btn-submit-pmt"]'
submit_scheduled_payment_xpath = '//*[@id="pmt-review-btn-submit-schedule"]'
pay_another_recipient_xpath = '//*[@id="app-body"]/div/div[3]/div[2]/div/div[3]/div/div/button[2]'
invoice_icon_xpath = '//*[@id="app-body"]/div/div[3]/div[2]/div/div[2]/div/ul/li[5]/div/span'
sign_in_with_different_email_xpath = '//*[@id="lr-password-link-change-user"]'

### Browser Launch

In [48]:
#Launch chrome driver
browser = webdriver.Chrome('/Users/victorwu/Downloads/chromedriver')

# Wait until sign in user id is clickable
# wait = WebDriverWait(browser, 60)

In [49]:
#Redirect to browser plastiq log in page
test_url = 'https://apps.payzinc.com/cardholder_ui/password'
main_url = 'https://apps.plastiq.com/cardholder_ui/start'
start_payment_url = 'https://apps.plastiq.com/cardholder_ui/payments/new/recipient'
browser.get(main_url)

In [50]:
for handler in logging.root.handlers[:]:
    logging.root.removeHandler(handler)

logging.basicConfig(filename='Debug.log',filemode = 'w', level=logging.INFO,
                   format='%(asctime)s [%(levelname)s]: %(message)s')

### Execute Script

In [53]:
for i in range(len(plastiq_email_list)):
    #Control for-loop
    if i==4:
        wait = WebDriverWait(browser, 180)
        plastiq_email = plastiq_email_list[i]
        plastiq_password = plastiq_password_list[i]
        
        #Log into Plastiq account
        plastiq_user_email = wait.until(EC.element_to_be_clickable((By.XPATH, plastiq_user_email_xpath)))
        plastiq_user_email.send_keys(plastiq_email_list[i])
        plastiq_user_email.send_keys(Keys.RETURN)
        logging.info('%s entered into email field', plastiq_email)

        plastiq_user_password = wait.until(EC.element_to_be_clickable((By.XPATH, plastiq_user_password_xpath)))
        plastiq_user_password.send_keys(plastiq_password_list[i])
        plastiq_user_password.send_keys(Keys.RETURN)
        logging.info('%s entered into password field', plastiq_password)
        
        payment_block = payroll_payments[payroll_payments['plastiq_email']==plastiq_email] 
        block_length = len(payment_block)
    
        for k in range(block_length):
            if k >=0:# and k <= 7:
                block = Block(payroll_payments, plastiq_email, k)
                logging.info('Start payment submission process for: [%s] %s' % (plastiq_email, block.payee_merchant_name))
                try:
                    start_payment_elem = get_elem(start_payment_xpath, 'START PAYMENT', 120)
                    try:
                        recipient_elem = get_elem("//span[contains(text(), '" + block.payee_merchant_name + "')]", 'SELECT RECIPIENT', 60)
                        recipient_elem.click()
                    except Exception as e:
                        #If unable to locate recipient, loop back
                        logging.error('Unable to locate %s due to %s' % (block.payee_merchant_name, e))
                        continue 
                    
                    start_payment_elem.click()
                    logging.info('Click START PAYMENT')
                    
                    #Wait until next payment details is clickable
                    next_payment_details_elem = get_elem(payment_details_xpath, 'NEXT: PAYMENT DETAILS', 30)

                    arrival_date_elem = get_elem(arrival_date_xpath, 'CALENDAR', 5)
                    arrival_date_elem.click()
                    logging.info('Click CALENDAR')

                    calendar_date_elem = get_elem('//*[@id="pq-calendar-day-' + calendar_date + '"]/div[1]', 'CALENDAR DATE', 5)
                    calendar_date_elem.click()
                    logging.info('Click CALENDAR DATE')

                    payment_amount_elem = get_elem(payment_amount_xpath, 'PAYMENT AMOUNT', 5)
                    send_keys(payment_amount_elem, block.amount)

                    #Nested inside try statement to check if 'does this payment repeat' prompt shows up    
                    try:
                        repeats_dropdown_elem = browser.find_element_by_xpath(schedule_dropdown_xpath)
                        repeats_dropdown_elem.click()
                        logging.info('Click REPEATS DROPDOWN')
                        one_time_payment_elem = browser.find_element_by_xpath(one_time_schedule_xpath)
                        one_time_payment_elem.click()
                        logging.info('Click ONE TIME PAYMENT')
                        next_payment_details_elem.click()
                        logging.info('Click NEXT PAYMENT DETAILS')
                    except:
                        logging.info('No DOES THIS PAYMENT REPEAT option available')
                        next_payment_details_elem.click()
                        logging.info('Click NEXT PAYMENT DETAILS')

                    account_holder_name_elem = get_elem(accountholder_name_xpath, 'ACCT HOLDER NAME', 10)
                    send_keys(account_holder_name_elem, block.account_holder_name)

                    invoice_number_elem = get_elem(invoice_number_xpath, 'INVOICE LINE', 5)
                    send_keys(invoice_number_elem, invoice_line)

                    memo_elem = get_elem(memo_xpath, 'MEMO LINE', 5)
                    send_keys(memo_elem, 'NA')

                    continue_payment_elem = get_elem(continue_payment_xpath, 'CONTINUE PAYMENT', 5)
                    continue_payment_elem.click()
                    logging.info('Click CONTINUE PAYMENT')
                    
                    try:
                        invoice_continue_elem = get_elem(continue_payment_xpath, 'CONTINUE PAYMENT', 10)
                        time.sleep(3)
                        try:
                            invoice_elem = browser.find_element_by_xpath('//*[@id="body"]/label/input')
                            logging.info('%s element variablized' % ('UPLOAD INVOICE'))
                            #wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="body"]/label/input')))
                            #browser.find_element_by_xpath('//*[@id="body"]/label/input')
                        except Exception as e:
                            logging.error('Failed to find %s: %s' % ('UPLOAD INVOICE', e))
                        #Upload invoice
                        invoice_elem.send_keys(os.getcwd()+"/EZ Messenger.png")

                        get_elem(invoice_upload_success_xpath, 'INVOICE UPLOAD SUCCESSFUL', 30)
                        
                        #Click continue
                        invoice_continue_elem = get_elem(continue_payment_xpath, 'CONTINUE PAYMENT', 10)
                        invoice_continue_elem.click()
                        logging.info('Click INVOICE CONTINUE')
                        
                        #SCROLL ALL THE WAY UP BEFORE GET ELEM ON SUBMIT PAYMENT...
                        submit_payment(submission_type)
                        
                        #Wait until success prompt shows up
                        pay_another_recipient_elem = get_elem(pay_another_recipient_xpath, 'PAY ANOTHER RECIPIENT', 120)
                        pay_another_recipient_elem.click()
                        logging.info('Click PAY ANOTHER RECIPIENT')

                        #Log in payment submission
                        payroll_payments['is_payment_submitted'][block.starting_index + k] = 1
                        
                    except:
                        logging.info('INVOICE UPLOAD step not found')
                        
                        submit_payment(submission_type)
                        
                        pay_another_recipient_elem = get_elem(pay_another_recipient_xpath, 'PAY ANOTHER RECIPIENT', 60)
                        pay_another_recipient_elem.click()
                        logging.info('Click PAY ANOTHER RECIPIENT')
                except Exception as e:

                    logging.error('[%s] %s: %s' % (plastiq_email, block.payee_merchant_name, e))
        
        browser.get(main_url)
        sign_in_with_different_email_elem = get_elem(sign_in_with_different_email_xpath, 'SIGN IN WITH DIFFERENT EMAIL', 60)
        sign_in_with_different_email_elem.click()
        logging.info('Click SIGN IN WITH DIFFERENT EMAIL')


/Users/victorwu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:110: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


TypeError: get_elem() missing 1 required positional argument: 'wait'

In [54]:
payroll_payments[payroll_payments['plastiq_email']=='ezplastiq05@yahoo.com']

,recipient_name,routing_number,account_number,type,amount,street_address_1,street_address_2,city,state,postal_code,country,phone,unique_string,recipient_id,plastiq_email,plastiq_password,payee_merchant_name,is_recipient_in_db,is_payment_submitted
397,Steven Rice,291070001,6443908571,1,2162.00,9517 Portland Ave.,NaN,Bloomington,MN,55420,US,6127079897,Steven Rice2910700016443908571,398,ezplastiq05@yahoo.com,EZplastiq05,[000398] Steven Rice,1,0
398,Steven Sanchez,314088556,706965100,1,311.38,218 Latch,NaN,San Antonio,TX,78213,US,2103962998,Steven Sanchez314088556706965100,399,ezplastiq05@yahoo.com,EZplastiq05,[000399] Steven Sanchez,1,0
399,Suzette George,322271627,310390593,1,65.00,1812 W Burbank Blvd #45,NaN,Burbank,CA,91506,US,8183986109,Suzette George322271627310390593,401,ezplastiq05@yahoo.com,EZplastiq05,[000401] Suzette George,1,1
400,T N T Legal Service,111301122,42110047554,1,1051.91,204 S. Caddo St.,NaN,Cleburne,TX,76031,US,8172028750,T N T Legal Service11130112242110047554,403,ezplastiq05@yahoo.com,EZplastiq05,[000403] T N T Legal Service,1,0
401,TABITHA COLVIN,082902757,502381304,1,225.00,2005 SUNSET RIDGE,NaN,CAVE CITY,AR,72521,US,8702832584,TABITHA COLVIN082902757502381304,404,ezplastiq05@yahoo.com,EZplastiq05,[000404] TABITHA COLVIN,1,0
402,TARA ENGSTROM,275976655,1000149227,1,420.00,5985 120TH AVENUE,NaN,ECHO,MN,56237,US,5078223484,TARA ENGSTROM2759766551000149227,405,ezplastiq05@yahoo.com,EZplastiq05,[000405] TARA ENGSTROM,1,0
403,TIFFANY ALLISON,273073958,198196,1,200.00,2227 E 29TH STREET,NaN,DES MOINES,IA,50317,US,5159885468,TIFFANY ALLISON273073958198196,406,ezplastiq05@yahoo.com,EZplastiq05,[000406] TIFFANY ALLISON,1,0
404,TIMOTHY YANCY,322172797,7556137,1,499.00,2360 N. Azurite Cir.,NaN,Mesa,AZ,85207,US,4808181341,TIMOTHY YANCY3221727977556137,408,ezplastiq05@yahoo.com,EZplastiq05,[000408] TIMOTHY YANCY,1,0
405,Tammy Aston,111900659,7148222016,1,315.01,P.O. Box 122042,NaN,Fort Worth,TX,76121,US,8174756094,Tammy Aston1119006597148222016,410,ezplastiq05@yahoo.com,EZplastiq05,[000410] Tammy Aston,1,0
406,Tammy Macz,111900659,2035977145,1,618.76,31248 North Head Drive,NaN,Spring,TX,77386,US,8322079038,Tammy Macz1119006592035977145,411,ezplastiq05@yahoo.com,EZplastiq05,[000411] Tammy Macz,1,0


In [ ]:
payroll_payments = payrol